# Mortgage file to SQL database

In [2]:
import sqlite3
import pandas as pd
import geopandas as gpd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# Atlantic Hurricane Dataset from 1979
https://www.nhc.noaa.gov/data/#hurdat

In [3]:
#Import data from url
hurricane_data = []
with open("../../Data/nhc.noaa.gov_data_hurdat_hurdat2-1851-2020-052921.txt", "r") as f:
    lines = f.readlines()
    storm_data = None
    for line in lines:
        parts = line.split(",")
        
        if len(parts) == 4 and int(parts[0][-4:]) >= 1979:  # Header Line
            #Get last 4 digits from first entry
            storm_data = {"id": parts[0].strip(), "name": parts[1].strip(), "entries": int(parts[2]), "year": int(parts[0][-4:])}
        else:  # Data Line
            hurricane_data.append({**storm_data, **{
                "date": parts[0].strip(),
                "time": parts[1].strip(),
                "record_id": parts[2].strip(),
                "status": parts[3].strip(),
                "latitude": parts[4].strip(),
                "longitude": parts[5].strip(),
                "max_wind": int(parts[6].strip()),
                "min_pressure": int(parts[7].strip()) if parts[12].strip() != "" else None
            }})

# Create DataFrame from parsed data
hurricane_df = pd.DataFrame(hurricane_data)
#convert date to datetime format
hurricane_df['date'] = pd.to_datetime(hurricane_df['date'], format='%Y%m%d')
#add month column to hurrican_df
hurricane_df['month'] = hurricane_df['date'].dt.month
#Save as csv
#hurricane_df.to_csv('../Data/hurricane_df.csv')

#describe data and make entries column the unique count of id
display(hurricane_df.describe())
display(hurricane_df)

,entries,year,date,max_wind,min_pressure,month
count,19180.000000,19180.000000,19180,19180.000000,19180.000000,19180.000000
mean,40.353910,2001.466632,2002-02-23 12:13:21.835245184,48.399791,881.660740,8.678311
min,2.000000,1979.000000,1979-06-11 00:00:00,-99.000000,-999.000000,1.000000
25%,24.000000,1991.000000,1991-10-30 00:00:00,30.000000,982.000000,8.000000
50%,38.000000,2003.000000,2003-08-27 12:00:00,40.000000,999.000000,9.000000
75%,55.000000,2012.000000,2012-05-26 00:00:00,60.000000,1006.000000,9.000000
max,96.000000,2020.000000,2020-11-18 00:00:00,165.000000,1024.000000,12.000000
std,20.312358,12.104994,NaN,25.734435,459.088771,1.400565


,id,name,entries,year,date,time,record_id,status,latitude,longitude,max_wind,min_pressure,month
0,AL011979,UNNAMED,22,1979,1979-06-11,1200,,TD,17.4N,81.1W,20,-999,6
1,AL011979,UNNAMED,22,1979,1979-06-11,1800,,TD,17.7N,80.8W,25,-999,6
2,AL011979,UNNAMED,22,1979,1979-06-12,0000,,TD,18.1N,80.2W,25,-999,6
3,AL011979,UNNAMED,22,1979,1979-06-12,0600,,TD,18.4N,79.9W,25,-999,6
4,AL011979,UNNAMED,22,1979,1979-06-12,1200,,TD,19.0N,79.6W,25,-999,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19175,AL312020,IOTA,26,2020,2020-11-17,1200,,HU,13.7N,84.7W,75,965,11
19176,AL312020,IOTA,26,2020,2020-11-17,1800,,TS,13.7N,85.7W,55,988,11
19177,AL312020,IOTA,26,2020,2020-11-18,0000,,TS,13.8N,86.7W,40,1000,11
19178,AL312020,IOTA,26,2020,2020-11-18,0600,,TS,13.8N,87.8W,35,1005,11


In [ ]:
hurricane_df_agg = hurricane_df.groupby(['year', 'month']).agg({'id': 'nunique', 'max_wind': 'median', 'min_pressure': 'median', 'status': 'count'}).reset_index()
display(hurricane_df_agg.head())